<a href="https://colab.research.google.com/github/limontec/vm-video-downloader/blob/main/VMVideoDownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <center><h3><h2>VM Video Downloader</h2>Download Vimeo videos easy v0.1</h3></center><br>

#@markdown ---
#@markdown ####Settings

import os
import sys
import base64
import requests
import subprocess
from google.colab import files

from tqdm import tqdm
#from moviepy.editor import *
url = "Type the [master|playlist].json url"  # @param {type:"string"}
file_name = "output1" # @param {type:"string"}
download_to_pc = True #@param {type:"boolean"}
#@markdown ---
if 'master.json' in url:
    url = url[:url.find('?')] + '?query_string_ranges=1'
    url = url.replace('master.json', 'master.mpd')
    print(url)
    subprocess.run(['youtube-dl', url, '-o', file_name])
    sys.exit(0)

def downloadFile():
  try:
    files.download(file_name)
    print("If download does not start, download the file manually from the directory in the left corner.")
  except FileNotFoundError:
    print("File not found!")
  except:
    print("Something else went wrong, try download manually")

def download(what, to, base):
    print('saving', what['mime_type'], 'to', to)
    with open(to, 'wb') as file:
        init_segment = base64.b64decode(what['init_segment'])
        file.write(init_segment)

        for segment in tqdm(what['segments']):
            segment_url = base + segment['url']
            resp = requests.get(segment_url, stream=True)
            if resp.status_code != 200:
                print('not 200!')
                print(segment_url)
                break
            for chunk in resp:
                file.write(chunk)
    print('done')


file_name += '.mp4'
base_url = url[:url.rfind('/', 0, -26) + 1]
content = requests.get(url).json()

vid_heights = [(i, d['height']) for (i, d) in enumerate(content['video'])]
vid_idx, _ = max(vid_heights, key=lambda _h: _h[1])

audio_quality = [(i, d['bitrate']) for (i, d) in enumerate(content['audio'])]
audio_idx, _ = max(audio_quality, key=lambda _h: _h[1])

video = content['video'][vid_idx]
audio = content['audio'][audio_idx]
base_url = base_url + content['base_url']

video_tmp_file = 'video.mp4'
audio_tmp_file = 'audio.mp4'

download(video, video_tmp_file, base_url + video['base_url'])
download(audio, audio_tmp_file, base_url + audio['base_url'])

command = ["ffmpeg", "-i", audio_tmp_file, "-i", video_tmp_file, "-c", "copy", file_name]

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print(result.stdout)  # Exibe a saída do comando
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    print(f"Output error: {e.stderr}")

if(download_to_pc):
    downloadFile()

os.remove(video_tmp_file)
os.remove(audio_tmp_file)

#@markdown ####How to find the playlist.json?
#@markdown #####Right click on the video > Inspect > Click on the Network tab > Play the video > Right click on "playlist.json?omit=..." > Copy the Request URL, something like this: vod-adaptive-ak.vimeocdn.com/exp=...


#@markdown ---
#@markdown #####Inspired by
#@markdown ##### https://gist.github.com/alexeygrigorev/a1bc540925054b71e1a7268e50ad55cd?permalink_comment_id=5097364#gistcomment-5097364

In [ ]:
#@markdown ####Run this to download all downloaded videos
#@markdown #####Use this only if you didn't marked "download_to_pc" option
def download_all_mp4_files():
    files_in_dir = os.listdir('.')
    mp4_files = [f for f in files_in_dir if f.endswith('.mp4')]

    for file_name in mp4_files:
        files.download(file_name)
        print(f"Downloading {file_name}...")

download_all_mp4_files()